In [1]:
import json
import pandas as pd
import os

PROFILING_DIR = '../consegna/profilings/GPT-4o'
INPUT_DIR = '../consegna/LLMAGENTS_CLAIMS'

Load profilings csv into dataframes

In [2]:
value_profilings = pd.read_csv(os.path.join(PROFILING_DIR, 'VALUE_PROFILING.csv'))
value_profilings = value_profilings.map(lambda x: x.lower() if isinstance(x, str) else x)
print(value_profilings)

AttributeError: 'DataFrame' object has no attribute 'map'

Load claim and convert it to dataframes

In [6]:
def load_json_as_df(filename):
    with open(os.path.join(INPUT_DIR, filename), 'r') as f:
        json_data = json.load(f)
    rows = []
    for key, entry in json_data.items():
        row = {}
        row['claim_id'] = key
        # Extract specifications
        for spec_key, spec in entry["specifications"].items():
            row[spec["name"]] = spec["value"]
        # Add measure and outcome
        row["Measure"] = entry["Measure"]
        row["Outcome"] = entry["Outcome"]
        rows.append(row)
    claim_df = pd.DataFrame(rows)
    filename = filename.replace("_claims.json","")
    claim_df['paper_id'] = filename.split('_')[0]
    claim_df['table_id'] = filename.split('_')[1]
    return claim_df

reassign values in dataframe

In [7]:
claims_df = []
for filename in os.listdir(INPUT_DIR):
    df = load_json_as_df(filename)
    print(df)
    claims_df.append(df)


   claim_id     Model Name Setting     Task     Measure Outcome    paper_id  \
0         0  GPT-3.5-turbo    Zero  OpenAGI  CLIP Score     0.0  2304.04370   
1         1  GPT-3.5-turbo     Few  OpenAGI  CLIP Score     0.0  2304.04370   
2         2       Claude-2    Zero  OpenAGI  CLIP Score     0.0  2304.04370   
3         3       Claude-2     Few  OpenAGI  CLIP Score  0.2543  2304.04370   
4         4          GPT-4    Zero  OpenAGI  CLIP Score     0.0  2304.04370   
5         5          GPT-4     Few  OpenAGI  CLIP Score  0.3055  2304.04370   
6         6  GPT-3.5-turbo    Zero  OpenAGI  BERT Score  0.1914  2304.04370   
7         7  GPT-3.5-turbo     Few  OpenAGI  BERT Score   0.382  2304.04370   
8         8       Claude-2    Zero  OpenAGI  BERT Score  0.2111  2304.04370   
9         9       Claude-2     Few  OpenAGI  BERT Score  0.5038  2304.04370   
10       10          GPT-4    Zero  OpenAGI  BERT Score  0.2076  2304.04370   
11       11          GPT-4     Few  OpenAGI  BERT Sc

In [43]:
with open('alignment_name_schema.json', 'r') as file:
    align_name = json.load(file)

with open('alignment_values_schema.json', 'r') as file:
    align_value = json.load(file)

aligned_df = pd.DataFrame(columns=align_name.keys())

excluded_columns = ['outcome', 'table_id', 'paper_id', 'claim_id']

for claim_df in claims_df:
    for _, row in claim_df.iterrows():
        aligned_row = {key: None for key in align_name.keys()}
        aligned_row["outcome"] = row["Outcome"]
        aligned_row["table_id"] = row["table_id"]
        aligned_row["paper_id"] = row["paper_id"]
        aligned_row["claim_id"] = row["claim_id"]
        for name_key, name_values in align_name.items():
            for col in [c for c in claim_df.columns if c not in excluded_columns]:
                if row[col].lower() in name_values:
                    for value_key, value_value in align_value.items():
                        if value_value is not None and row[col].lower() in value_value:
                            aligned_row[name_key] = value_key
                            break
                        else:
                            aligned_row[name_key] = row[col]
        aligned_df = pd.concat([aligned_df, pd.DataFrame([aligned_row])], ignore_index=True)

aligned_df.to_csv('result.csv')

JSON pe paulo  

In [30]:
def retrieve_spec_id(filename, claim_id, flag):
    with open(os.path.join(INPUT_DIR, filename), 'r') as f:
        data = json.load(f)
    
    for key, value in data[claim_id]['specifications'].items():
        if value['value'] == flag:
            return key

In [18]:
def constr_filename(paper_id, table_id):
    return f'{paper_id}_{table_id}_claims.json'

In [45]:
from pprint import pprint
def build_json(df):
    json_data = {}
    name2spec = {}
    for col in df.columns:
        if col not in ['claim_id', 'table_id', 'paper_id', 'Outcome']:
            nomi = []
            for _,row in df.iterrows():
                filename = constr_filename(row['paper_id'], row['table_id'])
                spec_id = retrieve_spec_id(filename, row['claim_id'], row[col])
                if (spec_id is not None):
                    nomi.append(f"{row['paper_id']}_{row['table_id']}_{row['claim_id']}_{spec_id}")  
            name2spec[col] = nomi
        json_data['aligned_names'] = name2spec
    return json_data

pprint(build_json(aligned_df))


{'aligned_names': {'agent configuration': ['2403.04783_4_0_0', '2403.04783_4_1_0', '2403.04783_4_2_0', '2403.04783_4_3_0', '2403.04783_4_4_0', '2403.04783_4_5_0'], 'attack method': ['2403.04783_1_0_0', '2403.04783_1_1_0', '2403.04783_1_2_0', '2403.04783_1_3_0', '2403.04783_1_4_0', '2403.04783_1_5_0', '2403.04783_1_6_0', '2403.04783_1_7_0', '2403.04783_1_8_0', '2403.04783_1_9_0', '2403.04783_1_10_0', '2403.04783_1_11_0', '2403.04783_1_12_0', '2403.04783_1_13_0', '2403.04783_1_14_0', '2403.04783_1_15_0', '2403.04783_1_16_0', '2403.04783_1_17_0', '2403.04783_1_18_0', '2403.04783_1_19_0'], 'attribute': ['2406.12707_3_0_0', '2406.12707_3_1_0', '2406.12707_3_2_0', '2406.12707_3_3_0', '2406.12707_3_4_0', '2406.12707_3_5_0', '2406.12707_3_6_0', '2406.12707_3_7_0', '2406.12707_3_8_0', '2406.12707_3_9_0', '2406.12707_3_10_0', '2406.12707_3_11_0', '2406.12707_3_12_0', '2406.12707_3_13_0', '2406.12707_3_14_0', '2406.12707_3_15_0', '2406.12707_3_16_0', '2406.12707_3_17_0', '2406.12707_3_18_0', '240

Values alignment

In [37]:
def extract_values_from_json(json_data):
    values_set = set()

    def extract_values(obj):
        if isinstance(obj, dict):
            for value in obj.values():
                extract_values(value)
        elif isinstance(obj, list):
            for item in obj:
                extract_values(item)
        else:
            values_set.add(obj)

    extract_values(json_data)
    return values_set

# Load the JSON file
with open('alignment_schema.json', 'r') as file:
    json_data = json.load(file)

# Extract values
values_set = extract_values_from_json(json_data)
sorted_list = sorted(list(values_set))
output_dict = {key: None for key in sorted_list}


output_file = "alignment_values_schema.json"
with open(output_file, "w") as json_file:
    json.dump(output_dict, json_file, indent=4)

